## ライブラリ読み込み

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import os, sys, gc, time, warnings, pickle, psutil, random
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from multiprocessing import Pool

warnings.filterwarnings('ignore')

## 各種パラメータ設定

In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
SEED = 5678                      # We want all things
seed_everything(SEED)            # to be as deterministic 

TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913+28            # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

### ファイルのパスは適宜変更してください ### 

# 読み込みが必要なpklファイルのパス
LAGS     = './lags_df_1_dbd.pkl'

# 読み込みが必要なcsvファイルのパス
EVALUATION   = './sales_train_evaluation.csv'
CALENDAR_CSV = './calendar.csv'
PRICE_CSV    = './sell_prices.csv'
SAMPLE_CSV   = './sample_submission.csv'

# WRMSSE計算用pklへのパス
SW_DF    = './sw_df_update.pkl'
ROLL_MAT = './roll_mat_df_update.pkl'

# 店舗別マスタが格納されているフォルダへのパス
GRID_PATH = "./"

# CVを行う区切りのリスト
# SPLIT_LIST = [[1858, 1885], [1886, 1913], [1914, 1941]]
SPLIT_LIST = [[1942, 1969]]

iteration_list = pd.read_pickle("./rounds_dict_wrmsse.pkl")

In [21]:
lgb_params = {
                'boosting_type': 'gbdt',
                'objective': 'poisson',
                'tweedie_variance_power': 1.1,
                'metric': 'rmse',
                'subsample': 0.5,
                'subsample_freq': 1,
                'learning_rate': 0.03,
                'num_leaves': 2**11-1,
                'min_data_in_leaf': 2**12-1,
                'feature_fraction': 0.5,
                'max_bin': 100,
                'n_estimators': 5000,
                'boost_from_average': False,
                'verbose': -1,
            } 

In [8]:
categorical_feat = [
    "event_name_1", "tm_wm", "tm_dw", "tm_w_end", "tm_y",
]

## 学習用関数定義

In [9]:
def merge_df(original_df, feat_path, feat_names=None, merge_key=None, day=0) -> pd.DataFrame:
    """
    パスとキーを指定して、特徴量を追加する   
    - original_df : 特徴を追加するDataFrame
    - feat_path   : 追加する特徴(pkl)のパス
    - feat_names  : pklの中で、結合する特徴を絞り込みたい場合に指定する
    - merge_key   : pklをmergeで結合する場合、キーを指定する(Noneの場合concatする)
    - day         : ラグ変数を結合する際にずらす日数(1日後予測モデルはday=1を引数として与える)
    
    """
    tmp = pd.read_pickle(feat_path)
    
    if day > 1:
        tmp["d"]= tmp["d"] + (day-1)
    
    if feat_names is not None: 
        tmp = tmp[feat_names]
        
    if merge_key is None:
        tmp = tmp[tmp.index.isin(original_df.index)]
        original_df = pd.concat([original_df, tmp], axis=1)
    else:
        original_df = pd.merge(original_df, tmp, on=merge_key, how="left")
    
    return original_df

In [10]:
def get_data_by_store(store, day=0):
    """
    store_idを指定して、学習用のデータフレームを取得する
    """  
    # 特徴結合済みの店舗別データフレームを読み込む
    pickle_name = GRID_PATH + "grid_df_master_"+store+".pkl"
    df = pd.read_pickle(pickle_name)
    
    # ラグ特徴(LAGS)を結合する
    lag = pd.read_pickle(LAGS)
    lag_features = list(lag.columns[3:])
    df = merge_df(df, feat_path=LAGS, feat_names=["id", "d"] + lag_features, merge_key=["id", "d"], day=day)
        
    # ラベルエンコーディングする
    le = LabelEncoder()
    df["tm_y"] = le.fit_transform(df["tm_y"])
    df["sell_price_state"] = le.fit_transform(df["sell_price_state"])
    
    # 特徴リストの生成(不要な特徴は除外する)
    remove_features = ['id','state_id','store_id','date','wm_yr_wk','d', TARGET]+["event_type_1", "event_name_2", "event_type_2"]
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # START_TRAINの値を元に、前半のデータを取り除く
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    # 店舗IDごとの細かい期間指定
    if store=="WI_1":
        df = df[df["d"]>=637].reset_index(drop=True)
    if store=="WI_3":
        df = df[df["d"]>=455].reset_index(drop=True)
    
    return df, features

In [11]:
STORES_IDS = pd.read_csv(EVALUATION)['store_id']
STORES_IDS = list(STORES_IDS.unique())

## WRMSSE計算の準備

In [12]:
sw_df = pd.read_pickle(SW_DF)
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

roll_mat_df = pd.read_pickle(ROLL_MAT)
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)

STORES_IDS = pd.read_csv(EVALUATION)['store_id']
STORES_IDS = list(STORES_IDS.unique())

# ★店舗別のSWを取得
SW_store = {}
for i, store_id in enumerate(STORES_IDS):
    SW_store[store_id] = SW[pd.Index(np.arange(42840))[roll_index.map(lambda x: (x[0] in [2, 7, 8, 11]) and store_id in x[1])]]

# ★店舗別のroll_matを取得
roll_mat_csr_store = {}
for i, store_id in enumerate(STORES_IDS):
    roll_mat_csr_store[store_id] = roll_mat_csr[pd.Index(np.arange(42840))[roll_index.map(lambda x: (x[0] in [2, 7, 8, 11]) and store_id in x[1])], i*3049:(i+1)*3049]
del roll_mat_df

In [13]:
# ★★店舗別にWRMSSEをレベル別で算出する関数を返す関数
def get_wrmsse_per_level_func(store_id, sell_price, level=None):
    def _wrmsse(preds, y_true, score_only=True, s = S, w = W, sw=SW_store):
        # Function to do quick rollups:
        def _rollup(v, store_id):
            '''
            v - np.array of size (30490 rows, n day columns)
            v_rolledup - array of size (n, 42840)
            '''
            v = v.reshape(28, 3049).T
            return roll_mat_csr_store[store_id]*v #(v.T*roll_mat_csr.T).T

        '''
        preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
        y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
        sequence_length - np.array of size (42840,)
        sales_weight - sales weights based on last 28 days: np.array (42840,)
        '''
        preds = preds / sell_price
        y_true = y_true / sell_price
        if score_only:
            scores = np.sqrt(
                        np.mean(
                            np.square(_rollup(preds-y_true, store_id))
                                ,axis=1)) * sw[store_id] #<-used to be mistake here
            if level is None:
                return np.sum(scores, axis=0)
            
            score_spans = [
                (0, 1), (1, 4), (4, 11), (11, 3060)          
            ]
            score = np.sum(scores[score_spans[level][0]:score_spans[level][1]], axis=0)
            return score
        else: 
            score_matrix = (np.square(_rollup(preds-y_true)) * np.square(w)[:, None])/ s[:, None]
            score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
            return score, score_matrix

    return _wrmsse

def get_wrmsse_per_level_metric(store_id, sell_price, level=None):
    wrmsse = get_wrmsse_per_level_func(store_id, sell_price, level=level)
    def wrmsse_metric(preds, data):

        # this function is calculate for last 28 days to consider the non-zero demand period

        # actual obserbed values / 正解ラベル
        y_true = data.get_label()
        
        if level is None:
            return 'wrmsse-store', wrmsse(preds, y_true), False
        
        return 'wrmsse-level' + str(level), wrmsse(preds, y_true), False
    return wrmsse_metric

In [14]:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T

# Function to calculate WRMSSE:
def wrmsse_metric(preds, y_true, score_only=False, s = S, w = W, sw=SW, level=None):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        scores = np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw
        if level is None:
            return np.sum(scores)/12
        
        score_spans = [
            (0, 1), (1, 4), (4, 14), (14, 17), (17, 24), (24, 33),
            (33, 54), (54, 84), (84, 154), (154, 3203), (3203, 12350), (12350, 42840)           
        ]
        score = np.sum(scores[score_spans[level][0]:score_spans[level][1]], axis=0)/12
        return score
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix

In [15]:
NUM_ITEMS = 30490
DAYS_PRED = 28
def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
    
          
    # train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    train = roll_mat_csr*np.c_[reshaped_preds, reshaped_true]
    """
    score = np.sum(
                np.sqrt(
                    np.mean( # ★ RMSSEの分子 1/h * SUM_t=n+1~n+h (Y_t - Y^_t)^2 に対応
                        np.square(
                            train[:,:num_col] - train[:,num_col:]) # ★ 予測誤差（Y_t - Y^_t）
                        ,axis=1)
                    
                    / weight1) * weight2)
    """
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(train[:,:num_col] - train[:,num_col:]))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    return 'wrmsse', score, False

## LGBMの学習(TS3-fold)

In [16]:
lgb_params['seed'] = SEED

In [17]:
STORES_IDS = pd.read_csv(EVALUATION)['store_id']
STORES_IDS = list(STORES_IDS.unique())

In [ ]:
%%time

result_list = []
validation_score = []
best_iteration_list = []

for val_range in SPLIT_LIST:
    print("------------------------------------------------------")
    print("train before : " + str(val_range[0]))
    print("------------------------------------------------------")
    
    # Create Dummy DataFrame to store predictions
    all_preds = pd.DataFrame() # 28日間のvalidation計算用
    sto_preds = pd.DataFrame() # store毎のvalidation計算用

    ######## store_id毎にモデルを学習する ########
    for store_id in STORES_IDS:      
        for day in range(1, 28+1):
            print('Train', store_id, f'{day}日後予測モデル')
            
            # store_idを指定して、特徴を結合済みのデータフレームを取得する
            grid_df, feature_columns = get_data_by_store(store_id, day=day)
            
            # CA_2については一部外れ値ぽいデータがあるので、validationに含まれなければ取り除く
            if val_range[0]>1878 and store_id=="CA_2":
                grid_df = grid_df[grid_df["d"]!=1877]
                grid_df = grid_df[grid_df["d"]!=1878]
                
            # TARGETの変更
            grid_df[TARGET] = grid_df[TARGET]*grid_df["sell_price"]

            train_mask = ((grid_df['d']>=START_TRAIN) & (grid_df['d']<=val_range[0]-1)) 
            valid_mask = ((grid_df['d']>val_range[0]-1-28) & (grid_df["d"]<=val_range[1]))
            valid_mask2 = ((grid_df['d']>val_range[0]-1-28) & (grid_df["d"]<=val_range[1]-28))
            
            df_train = grid_df[train_mask][feature_columns]
            df_valid = grid_df[valid_mask2][feature_columns]

            lgb_params["n_estimators"] = iteration_list[store_id][day-1]
            print("n_estimators", lgb_params["n_estimators"])
            
            train_data = lgb.Dataset(df_train, 
                                     label=grid_df[train_mask][TARGET], 
                                     categorical_feature=categorical_feat)
            valid_data = lgb.Dataset(df_valid,
                                     label=grid_df[valid_mask2][TARGET],
                                     categorical_feature=categorical_feat) 

            estimator = lgb.train(lgb_params,
                                  train_data,
                                  valid_sets = [valid_data],
                                  verbose_eval = 100,
                                  )

            valid_result_df = grid_df[valid_mask][feature_columns + ["d", "id"]]
            for_predict_df  = grid_df[valid_mask][feature_columns]
            
            # iterationを保存
            best_iteration_list.append(estimator.best_iteration)

            # この状態で保存
            pred = estimator.predict(for_predict_df)
            valid_result_df["pred"] = pred
            valid_result_df[["d", "id", "pred"]].to_pickle(f"./save3/save_{store_id}_day{day}_{val_range[0]}.pkl")
            
            # TARGETの値を元に戻す
            valid_result_df["pred"] = valid_result_df["pred"]/valid_result_df["sell_price"]
            
            # {day}日後の予測結果をデータフレームに結合する
            for PREDICT_DAY in range(day, day+1):
                day_to_predict = val_range[0]-1+PREDICT_DAY
                tmp = valid_result_df[valid_result_df["d"]==day_to_predict].rename(columns={"pred":"pred_" + str(day_to_predict)})
                tmp = tmp[["id", "pred_" + str(day_to_predict)]]

                if 'id' in list(sto_preds):
                    sto_preds = sto_preds.merge(tmp, on=['id'], how='left')
                else:
                    sto_preds = tmp.copy()
                    
            sto_preds.to_pickle(f"sto_preds_{store_id}_day{day}_{val_range[0]}.pkl")
    
#         # ★予測結果を日付でソート
#         cols = ["pred_" + str(x) for x in np.arange(val_range[0], val_range[1]+1)]
#         sto_preds = sto_preds[["id"] + cols]

#         # Ground truth:
#         sales =  pd.read_csv(EVALUATION)
#         dayCols = ["d_{}".format(i) for i in range(val_range[0], val_range[1]+1)]
#         y_true = sales[sales["store_id"] == store_id][dayCols]

#         valid_preds = sto_preds.drop("id", axis=1)

#         # ★店舗別にWRMSSEの値を算出
#         store_score = store_wrmsse(valid_preds.values, y_true.values, score_only=True)

#         print(f"{store_id} wrmsse: {store_score}")

        # 店舗別の出力を集約していく
        if 'id' in list(all_preds):
            all_preds = pd.concat([all_preds, sto_preds])
        else:
            all_preds = sto_preds.copy()
            
        sto_preds = pd.DataFrame() # store毎の出力をリセット
    
    print("------------------------------------------------------")
    print("---------------- calc CV score -----------------------")
    print("------------------------------------------------------")

    # Predict
    sales =  pd.read_csv(EVALUATION)
    base_id_list = sales["id"].to_list()
    all_preds_val = pd.DataFrame({"id":base_id_list})
    all_preds_val = pd.merge(all_preds_val, all_preds, on="id", how="left")
    all_preds_val = all_preds_val.fillna(0)


------------------------------------------------------
train before : 1942
------------------------------------------------------
Train TX_2 1日後予測モデル
n_estimators 850
[100]	valid_0's rmse: 7.21868
[200]	valid_0's rmse: 6.78152


In [ ]:
estimator.best_iteration